# A Guided Tour of Ray Core: Remote Objects

© 2019-2022, Anyscale. All Rights Reserved

### Learning objectives

In this tutorial, you learn about:
 * Ray Futures as one of the patterns
 * Ray's distributed Plasma object stores
 * How obejcts are stored and fetched from the distributed shared object store
 * Peek into nbject resolution protocol



[*Remote Objects*](https://docs.ray.io/en/latest/walkthrough.html#objects-in-ray)
implement a [*shared-memory object store*](https://en.wikipedia.org/wiki/Shared_memory) pattern.

Objects are immutable and can be accessed from anywhere on the cluster, as they are stored in the cluster shared memory. An object ref is essentially a pointer or a unique ID that can be used to refer to a remote object without seeing its value. If you’re familiar with futures, Ray object refs are conceptually similar.

<img src="images/shared_memory.png"  height="40%" width="65%">

In general, small objects are stored in their owner’s **in-process store** while large objects are stored in the **distributed object store**. This decision is meant to reduce the memory footprint and resolution time for each object. Note that in the latter case, a placeholder object is stored in the in-process store to indicate the object has been promoted to shared memory.

In the case if there is no space in the shared-memory, objects are spilled over to disk. But the main point here is that
shared-memory allows zero-copy to processes on the same worker node.

<img src="images/shared_memory_plasma_store.png"  height="40%" width="65%">

---

## 2. Object references as Futures Pattern

First, let's start Ray…

In [1]:
import logging
from pprint import pprint
import ray

if ray.is_initialized:
    ray.shutdown()
context = ray.init(logging_level=logging.ERROR)
pprint(context)

RayContext(dashboard_url='127.0.0.1:8266', python_version='3.8.13', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-19_16-11-02_184046_11147/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-19_16-11-02_184046_11147/sockets/raylet', 'webui_url': '127.0.0.1:8266', 'session_dir': '/tmp/ray/session_2022-07-19_16-11-02_184046_11147', 'metrics_export_port': 64970, 'gcs_address': '127.0.0.1:60936', 'address': '127.0.0.1:60936', 'node_id': 'ecfa1f5c927d43dac705d88ea8a14b9c51732282df0a9ef407d5b65b'})


In [2]:
print(f"Dashboard url: http://{context.address_info['webui_url']}")

Dashboard url: http://127.0.0.1:8266


## Remote Objects example

To start, we'll define a remote object...

In [3]:
%%time
num_list = [23, 42, 93]

# returns an objectRef
obj_ref = ray.put(num_list)
obj_ref

CPU times: user 33.7 ms, sys: 12.4 ms, total: 46.1 ms
Wall time: 47.2 ms


ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000001000000)

Then retrieve the value of this object reference. This follows an object resolution protocol.

<img src="images/object_resolution.png" height="40%" width="65%">

Small objects are resolved by copying them directly from the owner’s **in-process store**. For example, if the owner calls `ray.get`, the system looks up and deserializes the value from the local **in-process store**. If the owner submits a dependent task, it inlines the object by copying the value directly into the task description. Note that these objects are local to the owner process: if a borrower attempts to resolve the value, the object is promoted to shared memory, where it can be retrieved through the distributed object resolution protocol described next.

Resolving a large object. The object `x` is initially created on Node 2, e.g., because the task that returned the value ran on that node. This shows the steps when the owner (the caller of the task) calls `ray.get`: 

 1) Lookup object’s locations at the owner. 
 2) Select a location and send a request for a copy of the object. 
 3) Receive the object.



In [4]:
val = ray.get(obj_ref)
val

[23, 42, 93]

Let's combine use of a remote function with a remote object, to illustrate *composable futures*:

In [5]:
@ray.remote
def my_function (num_list):
    return sum(num_list)

In other words, the remote function `myfunction()` will sum the list of integers in the remote object `num_list`:

In [6]:
calc_ref = my_function.remote(obj_ref)

In [7]:
result = ray.get(calc_ref)
result

158

You can gather the values of multiple object references in parallel using comprehension:
 1. Each value is put in the object store and its `ObjRefID` is immediately returned
 2. The comprehsion constructs a list of `ObjRefIDs` for each element in the loop
 3. A final `get(list_obj_refs`) is invoked to fetch the list

In [8]:
results = ray.get([ray.put(i) for i in range(10)])
results

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Passing Objects by Reference

Ray object references can be freely passed around a Ray application. This means that they can be passed as arguments to tasks, actor methods, and even stored in other objects. Objects are tracked via distributed reference counting, and their data is automatically freed once all references to the object are deleted.

In [11]:
# Define a Task
@ray.remote
def echo(x):
    print(f"current value of x: {x}")
          
# Define an Actor
@ray.remote
class Echo:
    def __init__(self, x):
        self.x = x
    def update(self, new_x):
        self.x = new_x
    def echo(self):
        print(f"current value of x: {self.x}")

In [12]:
x = list(range(10))
obj_ref_x = ray.put(x)

### Pass-by-value

Send the object to a task as a top-level argument.
The object will be *de-referenced* automatically, so the task only sees its value.

In [13]:
echo.remote(obj_ref_x)

ObjectRef(16310a0f0a45af5cffffffffffffffffffffffff0100000001000000)

(echo pid=11510) current value of x: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [14]:
actor = Echo.remote(obj_ref_x)
actor.echo.remote()

ObjectRef(32d950ec0ccf9d2abc6507f09beab6f70104d1a60100000001000000)

(Echo pid=12604) current value of x: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


Update value with a new reference

In [15]:
x = [10, 25, 30]
obj_ref_x = ray.put(x)
actor.update.remote(obj_ref_x)
actor.echo.remote()

ObjectRef(f4402ec78d3a2607bc6507f09beab6f70104d1a60100000001000000)

(Echo pid=12604) current value of x: [10, 25, 30]


### Pass-by-reference

When passed inside a Python list or other data structure,
the *object ref is preserved*. The object data is not transferred to the worker
when it is passed by reference, until `ray.get()` is called on the reference.

You can pass by reference in two ways:
 1. as a dictionary `.remote({"obj": obj_ref_x})`
 2. as list of objRefs `.remote([obj_ref_x])`

In [16]:
x = list(range(20))
obj_ref_x = ray.put(x)
echo.remote({"obj": obj_ref_x})

ObjectRef(f91b78d7db9a6593ffffffffffffffffffffffff0100000001000000)

(echo pid=11510) current value of x: {'obj': ObjectRef(00ffffffffffffffffffffffffffffffffffffff010000000e000000)}


In [17]:
actor.update.remote([obj_ref_x])
actor.echo.remote()

ObjectRef(8849b62d89cb30f9bc6507f09beab6f70104d1a60100000001000000)

(Echo pid=12604) current value of x: [ObjectRef(00ffffffffffffffffffffffffffffffffffffff010000000e000000)]


## What about long running tasks?

Now let's set a timeout to return early from attempted access of a remote object that is blocking for too long...

In [18]:
import time

@ray.remote
def long_running_function ():
    time.sleep(10)
    return 42

You can control how long you want to wait for the task to finish

In [19]:
%%time

from ray.exceptions import GetTimeoutError

obj_ref = long_running_function.remote()

try:
    ray.get(obj_ref, timeout=6)
except GetTimeoutError:
    print("`get` timed out")

`get` timed out
CPU times: user 31.4 ms, sys: 19.6 ms, total: 51 ms
Wall time: 6.02 s


In [20]:
ray.shutdown()

### Exercises

1. Send a list of object references returned by `ray.put(x)` 
2. Use comprehension to construct this list and send it to `my_function.remote(list_of_object_refs)` to return the sum of the list
3. Create a python object, use `ray.put(pobj)` and `ray.get(pobj)`
4. Create large lists and python dictionaries, put them in object store.

### Homework

1. Read references to get advanced deep dives and more about Ray objects
2. [Serialization](https://docs.ray.io/en/latest/ray-core/objects/serialization.html)
3. [Memory Management](https://docs.ray.io/en/latest/ray-core/objects/memory-management.html)
4. [Object Spilling](https://docs.ray.io/en/latest/ray-core/objects/object-spilling.html)
5. [Fault Tolerance](https://docs.ray.io/en/latest/ray-core/objects/fault-tolerance.html)

## References

 * [Ray Architecture Reference](https://docs.google.com/document/d/1lAy0Owi-vPz2jEqBSaHNQcy2IBSDEHyXNOQZlGuj93c/preview#)
 * [Ray Internals: A peek at ray,get](https://www.youtube.com/watch?v=a1kNnQu6vGw)
 * [Ray Internals: Object management with Ownership Model](https://www.anyscale.com/events/2021/06/22/ray-internals-object-management-with-the-ownership-model)
 * [Deep Dive into Ray scheduling Policies](https://www.anyscale.com/events/2021/06/23/a-deep-dive-into-rays-scheduling-policy)
 * [Redis in Ray: Past and future](https://www.anyscale.com/blog/redis-in-ray-past-and-future)
 * [StackOverFlow: How Ray Shares Data](https://stackoverflow.com/questions/58082023/how-exactly-does-ray-share-data-to-workers/71500979#71500979)
 